# Project
**Jacob Medley**

The project should provide some useful functionality in science or engineering. It could be a command line utility, or a package to use in notebooks. There should be some substance, but it does not need to be extensive. I don't expect it should take more than a few hours to write the code. It is not necessary to write very sophisticated code. Overall the project should demonstrate you have learned something in this class.

1. The project must be pip installable
2. Your project should utilize git, and there should be a version history.
3. Your project should have some tests.
4. Your project should use at least one code quality tool.
5. Your project should have a readme.md and LICENSE file.
6. The code should be well documented.
7. The code should be original work.
8. You should push it to a GitHUB repo.

At the end of the mini you will give a live demonstration of your project and what it does.

 


The two main constraints would be mole and energy balance.

$ \frac{dN_j}{dt} = F_{j, in} - F_{j, out} + r_j V$  

$\frac{dT}{dt} = \frac{\dot{Q} - \dot{W}_s - \sum F_{i,0} C_{P_i} (T - T_{i0}) + [- \Delta H_{Rx} (T)(-r_A V)]}{\sum N_i C_{P_i}}$

The objective function to maximize is:  

$\text{X} = \frac{F_{j,in} - F_{j, out}}{F_{j, out}}$   

Where the species $j$ is $\text{C}_2 \text{H}_5 \text{O}_2 \text{C}_2 \text{H}_5$ as long as $\text{NaOH}$ is in excess.

Assumptions:  

- Elementary kinetics
- Shaft work is negligible. ($\dot{\text{W}_\text{s}}$ is 0)
- Obtective is maximizing conversion of reactants. 
- The three variables to be optimized are inlet flow of each species ($F_{C_2 H_5 O_2 C_2 H_5,\;in}$   
  and $F_{\text{NaOH}, \; in}$), the volume of the overall reactor ($V$), and the inlet temperature of the reactor ($T_{in}$).

## Objective  

The goal of this Python package is to optimize product output for a CSTR reactor with given input constraints. 

$\sum_{i=1}^{n} i$

In [ ]:
from pyomo.environ import ConcreteModel, Var, Constraint, Objective, SolverFactory, Piecewise, NonNegativeReals, exp, value, Param
from thermo import Chemical

def reaction_enthalpy(reactants, products):
        delta_H_rxn = 0.0
        for reactant, coefficient in reactants.items():
            compound = Chemical(reactant)
            delta_H_rxn -= coefficient * compound.Hf
        for product, coefficient in products.items():
            compound = Chemical(product)
            delta_H_rxn += coefficient * compound.Hf
        return delta_H_rxn

# Function to define specific heat using Piecewise
def specific_heat(compound_formula, model):
    compound = Chemical(compound_formula)
    breakpoints = [compound.Tm, compound.Tb, 10000]  # Extended range with large number
    values = [
        compound.HeatCapacitySolid(compound.Tm) / compound.MW,
        compound.HeatCapacityLiquid(compound.Tm) / compound.MW,
        compound.HeatCapacityGas(compound.Tb) / compound.MW
    ]
    SH = Var(domain=NonNegativeReals)

    # Define the function rule separately
    def f_rule(model, x):
        if x <= breakpoints[0]:
            return values[0]
        elif x <= breakpoints[1]:
            return values[1]
        else:
            return values[2]

    Piecewise(SH, model.T_in,
            pw_pts=breakpoints,
            f_rule=f_rule,
            pw_constr_type='EQ',  # yvar is equal to the piecewise function
            pw_repn='SOS2')  # Use SOS2 as the piecewise representation method
    return SH

def optimize_cstr(initial_conditions, rate_constant_298K, activation_energy, compound_names):
    # Create a new model
    model = ConcreteModel()

    # Constants
    R = 8.314  # Gas constant J/(mol K)

    # Define model variables
    model.F_A_in = Var(domain=NonNegativeReals, initialize=initial_conditions['F_A_in'])
    model.F_B_in = Var(domain=NonNegativeReals, initialize=initial_conditions['F_B_in'])
    model.F_C_in = 0#Param(initialize=0)
    model.F_D_in = 0#Param(initialize=0)
    model.F_A_out = Var(domain=NonNegativeReals)
    model.F_B_out = Var(domain=NonNegativeReals)
    model.F_C_out = Var(domain=NonNegativeReals, initialize=0)
    model.F_D_out = Var(domain=NonNegativeReals, initialize=0)
    model.V = Var(domain=NonNegativeReals)#, initialize=initial_conditions['V'])
    model.T_in = Var(domain=NonNegativeReals, initialize=initial_conditions['T_in'])
    model.T = Var(domain=NonNegativeReals, initialize=model.T_in)
    model.Q_dot = Param(initialize=0)
    model.W_dot = Param(initialize=0)


    # Parameters for reaction rate
    model.k0 = Param(initialize=rate_constant_298K)
    model.Ea = Param(initialize=activation_energy)

    for compound, formula in compound_names.items():
        setattr(model, f'SH_{compound}', specific_heat(formula, model))
    
    model.delta_H_rx = Param(initialize=reaction_enthalpy(
        reactants={'C4H8O2': 1, 'H2O': 1},  
        products={'C2H5OH': 1, 'C2H4O2': 1} 
    ))

    # Reaction rate expression using Arrhenius equation
    def reaction_rate_expression(model):
        return model.k0 * exp(-model.Ea / (R * model.T)) * (model.F_A_in/model.V) * (model.F_B_in/model.V)

    model.reaction_rate = Var(domain=NonNegativeReals)
    model.reaction_rate_constraint = Constraint(expr=model.reaction_rate == reaction_rate_expression(model))

    # Mole balance for ethyl acetate (j)
    def mole_balance_ruleA(model):
        return model.F_A_in - model.F_A_out + model.reaction_rate * model.V == 0
    model.mole_balance_A = Constraint(rule=mole_balance_ruleA)
    
    def mole_balance_ruleB(model):
        return model.F_B_in - model.F_B_out + model.reaction_rate * model.V == 0
    model.mole_balance_B = Constraint(rule=mole_balance_ruleB)

    # Mole balance for product C
    def mole_balance_ruleC(model):
        return model.F_C_in - model.F_C_out + model.reaction_rate * model.V == 0
    model.mole_balance_C = Constraint(rule=mole_balance_ruleC)

    # Mole balance for product D
    def mole_balance_ruleD(model):
        return model.F_D_in - model.F_D_out + model.reaction_rate * model.V == 0
    model.mole_balance_D = Constraint(rule=mole_balance_ruleD)

    def energy_balance_rule(model):
        # Reference the specific heat piecewise definitions
        Cp_A = model.SH_A
        Cp_B = model.SH_B
        Cp_C = model.SH_C
        Cp_D = model.SH_D

        # Heat capacity flow term for inlet streams (A and B)
        heat_capacity_flow_inlet = (model.F_A_in * Cp_A + model.F_B_in * Cp_B) * (model.T - model.T_in)
        # Heat capacity flow term for outlet streams (C and D)
        heat_capacity_flow_outlet = (model.F_C_out * Cp_C + model.F_D_out * Cp_D) * (model.T - model.T_in)
        # Reaction enthalpy term
        reaction_enthalpy_term = model.delta_H_rx * model.reaction_rate * model.V
        # Sum of molar flow rates times heat capacities
        total_molar_flow_heat_capacity = (model.F_A_out * Cp_A + model.F_B_out * Cp_B + model.F_C_out * Cp_C + model.F_D_out * Cp_D)

        # Energy balance equation set to zero for steady state
        return (model.Q_dot - model.W_dot - heat_capacity_flow_inlet + reaction_enthalpy_term) / total_molar_flow_heat_capacity == 0

    model.energy_balance = Constraint(rule=energy_balance_rule)


    # Objective function to maximize conversion X
    def objective_rule(model):
        return (model.F_A_in - model.F_A_out) / model.F_A_out
    model.objective = Objective(rule=objective_rule, sense=1)  # sense=1 for maximization

    # Define a solver and solve the model
    solver = SolverFactory('ipopt')
    results = solver.solve(model, tee=True)

    # Output the results
    if (results.solver.status == 'ok') and (results.solver.termination_condition == 'optimal'):
        print('Found optimal solution.')
        print('F_A_in:', value(model.F_A_in))
        print('V:', value(model.V))
        print('T:', value(model.T))
    else:
        print('No optimal solution found.')
    
    return model

compound_names = {
    'A': 'C4H8O2',  # Ethyl Acetate
    'B': 'H2O',     # Water
    'C': 'C2H5OH',  # Ethanol
    'D': 'C2H4O2'   # Acetic Acid
}

# Example usage of the optimization function
initial_conditions = {
    'F_A_in': 1.0,  # Inlet flow rate of ethyl acetate
    'F_B_in': 1.0, # Outlet flow rate of ethyl acetate, initial guess
    #'V': 10,        # Reactor volume
    'T_in': 298,    # Inlet temperature of the reactor
    #'T_0': 298      # Reference temperature
}
rate_constant_298K = 1.0e4  # Example rate constant at 298K
activation_energy = 500   # Example activation energy in J/mol

# Optimize the CSTR
CSTR_model = optimize_cstr(initial_conditions, rate_constant_298K, activation_energy, compound_names)


In [ ]:
from pyomo.environ import ConcreteModel, Var, Constraint, Objective, SolverFactory, NonNegativeReals, exp, value, Param
import numpy as np

def optimize_cstr(initial_conditions, rate_constant_298K, activation_energy):
    # Create a new model
    model = ConcreteModel()

    # Constants
    R = 8.314 # Gas constant J/(mol K)

    # Define model variables
    model.F_in = Var(domain=NonNegativeReals, initialize=initial_conditions['F_in'])
    model.F_out = Var(domain=NonNegativeReals)  # Initialize at 0 for the solver to find an optimum
    model.V = Var(domain=NonNegativeReals, initialize=initial_conditions['V'])
    model.T = Var(domain=NonNegativeReals, initialize=initial_conditions['T'])

    # Parameters for reaction rate
    model.k0 = Param(initialize=rate_constant_298K)
    model.Ea = Param(initialize=activation_energy)

    # Reaction rate expression using Arrhenius equation
    def reaction_rate_expression(model):
        return model.k0 * exp(-model.Ea / (R * model.T)) * model.F_in

    model.reaction_rate = Var(domain=NonNegativeReals, initialize=0)
    model.reaction_rate_constraint = Constraint(expr=model.reaction_rate == reaction_rate_expression(model))

    # Mole balance for reactants
    def mole_balance_rule(model):
        return model.F_out == model.reaction_rate * model.V
    model.mole_balance = Constraint(rule=mole_balance_rule)

    # Objective function to maximize the output flow
    def objective_rule(model):
        return model.F_out
    model.objective = Objective(rule=objective_rule, sense=1)  # sense=1 for maximization

    # Define a solver and solve the model
    solver = SolverFactory('ipopt')
    results = solver.solve(model, tee=True)

    # Check if the solver found an optimal solution
    if (results.solver.status == 'ok') and (results.solver.termination_condition == 'optimal'):
        print('Found optimal solution.')
        print('F_in:', value(model.F_in))
        print('F_out:', value(model.F_out))
        print('V:', value(model.V))
        print('T:', value(model.T))
    else:
        print('No optimal solution found.')
    
    return model

# Example usage of the optimization function
initial_conditions = {
    'F_A_in': 1.0,
    'F_B_in': 1.0,  # Inlet flow rate of reactant
    'V': 10,      # Reactor volume
    'T': 350      # Temperature
}
rate_constant_298K = 1.0e4  # Example rate constant at 298K
activation_energy = 50000   # Example activation energy in J/mol

# Optimize the CSTR
CSTR_model = optimize_cstr(initial_conditions, rate_constant_298K, activation_energy)


In [ ]:
from pyomo.environ import ConcreteModel, Var, Constraint, Objective, SolverFactory, NonNegativeReals, exp, value, Param
from thermo import Chemical
R = 8.314 # Gas constant J/(mol K)
Ea = 50000

def optimize_cstr(initial_conditions, rate_constant_298K, activation_energy, reactants, products):
    # Create a new model
    CSTR = ConcreteModel()

    # Constants
    R = 8.314 # Gas constant J/(mol K)
    
    # Define model variables
    CSTR.F_in_reactant = Var(reactants.keys(), domain=NonNegativeReals, initialize=initial_conditions['F_in_reactant'])
    CSTR.F_out_product = Var(products.keys(), domain=NonNegativeReals) # Initialize at 0 for the solver to find an optimum
    CSTR.V = Var(domain=NonNegativeReals, initialize=initial_conditions['V'])
    CSTR.T = Var(domain=NonNegativeReals, initialize=initial_conditions['T'])

    # Parameters for reaction rate
    CSTR.k0 = Param(initialize=rate_constant_298K)
    CSTR.Ea = Param(initialize=activation_energy)
    
    # Reaction rate expression using Arrhenius equation
    def reaction_rate_expression(model):
        return model.k0 * (Ea/(R * model.T)) * sum(model.F_in_reactant[comp] for comp in reactants.keys())
    
    CSTR.reaction_rate = Var(domain=NonNegativeReals, initialize=0)
    CSTR.reaction_rate_constraint = Constraint(expr=CSTR.reaction_rate == reaction_rate_expression(CSTR))
    
    # Mole balance for reactants and products
    def mole_balance_rule(model, component):
        if component in reactants:
            return CSTR.F_out_product[component] == -model.reaction_rate * reactants[component] * model.V
        elif component in products:
            return CSTR.F_out_product[component] == model.reaction_rate * products[component] * model.V
    CSTR.mole_balance = Constraint(reactants.keys() | products.keys(), rule=mole_balance_rule)
    
    # Energy balance for the reactor
    def energy_balance_rule(model):
        # For simplicity, assume the heat capacity is constant and the reaction is isothermal
        # Here, delta_H_rxn would be obtained from thermochemical data for the reaction at T
        delta_H_rxn = reaction_enthalpy(reactants, products, model.T)
        return 0 == -delta_H_rxn * model.reaction_rate * model.V
    CSTR.energy_balance = Constraint(rule=energy_balance_rule)
    
    # Objective function to maximize the production of products
    def objective_rule(model):
        # Sum of products
        return sum(CSTR.F_out_product[prod] for prod in products.keys())
    CSTR.objective = Objective(rule=objective_rule, sense=1)  # sense=1 for maximization

    # Define a solver and solve the model
    solver = SolverFactory('ipopt') # 'ipopt' is an open-source solver for nonlinear optimization
    results = solver.solve(CSTR, tee=True)  # 'tee=True' to display solver output

    # Check if the solver found an optimal solution
    if (results.solver.status == 'ok') and (results.solver.termination_condition == 'optimal'):
        print('Found optimal solution.')
        print('F_in_reactant:', {comp: value(CSTR.F_in_reactant[comp]) for comp in reactants.keys()})
        print('F_out_product:', {comp: value(CSTR.F_out_product[comp]) for comp in products.keys()})
        print('V:', value(CSTR.V))
        print('T:', value(CSTR.T))
    else:
        print('No optimal solution found.')
    
    return CSTR

# Define a function to calculate the reaction enthalpy at a given temperature
def reaction_enthalpy(reactants, products, T):
    # For demonstration, using a placeholder for reaction enthalpy value which typically depends on T
    # In practice, this would require more complex calculations involving thermochemical databases
    return -50 * T # Placeholder value representing an exothermic reaction

# Example usage of the optimization function
initial_conditions = {
    'F_in_reactant': 1.0,  # Inlet flow of reactant (adjust as per your need)
    'V': 10,              # Reactor volume
    'T': 350               # Temperature
}
rate_constant_298K = 1.0e4  # Example rate constant at 298K
activation_energy = 5000000    # Example activation energy in J/mol

# Define reactants and products for your specific reaction
reactants = {'CH3OH': 1}  # Example reactants
products = {'CH3COOCH3': 1, 'H2O': 1}  # Example products

# Optimize the CSTR
CSTR_model = optimize_cstr(initial_conditions, rate_constant_298K, activation_energy, reactants, products)


In [ ]:
from pyomo.environ import ConcreteModel, Var, Piecewise, Objective, SolverFactory, NonNegativeReals, exp, value, Constraint
from thermo import Chemical

def optimize_cstr(initial_conditions, rate_constant_298K, activation_energy, reactants_stoichiometry, products_stoichiometry, compound_formula):
    CSTR = ConcreteModel()

    # Define model variables
    CSTR.F_in = Var(domain=NonNegativeReals,initialize=initial_conditions['F_in'])
    CSTR.F_out = Var(domain=NonNegativeReals)
    CSTR.V = Var(domain=NonNegativeReals, initialize=initial_conditions['V'])
    CSTR.T_in = Var(domain=NonNegativeReals, initialize=initial_conditions['T_in'])
    CSTR.T_out = Var(domain=NonNegativeReals)

    # Function to define specific heat using Piecewise
    def specific_heat(compound_formula, model):
        compound = Chemical(compound_formula)
        breakpoints = [compound.Tm, compound.Tb, 10000]  # Extended range with large number
        values = [
            compound.HeatCapacitySolid(compound.Tm) / compound.MW,
            compound.HeatCapacityLiquid(compound.Tm) / compound.MW,
            compound.HeatCapacityGas(compound.Tb) / compound.MW
        ]
        SH = Var(domain=NonNegativeReals)

        # Define the function rule separately
        def f_rule(model, x):
            if x <= breakpoints[0]:
                return values[0]
            elif x <= breakpoints[1]:
                return values[1]
            else:
                return values[2]

        Piecewise(SH, model.T_in,
                pw_pts=breakpoints,
                f_rule=f_rule,
                pw_constr_type='EQ',  # yvar is equal to the piecewise function
                pw_repn='SOS2')  # Use SOS2 as the piecewise representation method
        return SH

    # Initialize specific heat for the compound
    CSTR.SH_C2H5O2C2H5 = specific_heat(compound_formula, CSTR)

    # Define the reaction enthalpy function within the main function
    def reaction_enthalpy(reactants, products):
        delta_H_rxn = 0.0
        for reactant, coefficient in reactants.items():
            compound = Chemical(reactant)
            delta_H_rxn -= coefficient * compound.Hf
        for product, coefficient in products.items():
            compound = Chemical(product)
            delta_H_rxn += coefficient * compound.Hf
        return delta_H_rxn

    # Define the reaction rate using the Arrhenius equation (assuming first-order kinetics)
    def reaction_rate(model):
        A_factor = rate_constant_298K * exp(-activation_energy / (8.314 * model.T_in))
        return A_factor * model.F_in

    # Define the energy balance constraint
    reaction_enthalpy_value = reaction_enthalpy(reactants_stoichiometry, products_stoichiometry)

    # Ensure the constraint uses the value at the current temperature
    CSTR.energy_balance_constraint = Constraint(expr=CSTR.SH * CSTR.F_in - reaction_enthalpy_value * reaction_rate(CSTR) * CSTR.V == 0)

    # Define the objective function to maximize the production of the product
    CSTR.objective = Objective(expr=CSTR.F_in - CSTR.F_out + reaction_rate(CSTR) * CSTR.V)#, sense=1)  # sense=1 for maximization

    # Solve the optimization problem using a suitable solver
    solver = SolverFactory('gurobi')
    #solver.options['mipgap'] = 0.01  # Optional: Set a gap tolerance if needed
    result = solver.solve(CSTR)#, tee=True, symbolic_solver_labels=True, keepfiles=False, mipgap=0.01)

    if result.solver.status == 'ok' and result.solver.termination_condition == 'optimal':
        print("Optimized Parameters:", value(CSTR.F_in), value(CSTR.V), value(CSTR.T_in))
    else:
        print("No optimal solution found.")

    return CSTR

# Example usage
initial_conditions = {
    'F_in': 1.0,  # Total inlet flow rate, arbitrary units
    'V': 100,    # Reactor volume, arbitrary units
    'T_in': 350   # Inlet temperature, K
}
rate_constant_298K = 0.000001  # Reaction rate constant at 298 K, arbitrary units
activation_energy = 50000  # Activation energy, J/mol
reactants_stoichiometry = {'C2H5O2C2H5': 1, 'NaOH': 1}
products_stoichiometry = {'C2H3O2Na': 1, 'C2H5OH': 1}
compound_formula = 'C2H5O2C2H5'

CSTR_model = optimize_cstr(initial_conditions, rate_constant_298K, activation_energy, reactants_stoichiometry, products_stoichiometry, compound_formula)

In [ ]:
from pyomo.environ import ConcreteModel, Var, Constraint, Objective, SolverFactory, NonNegativeReals, exp, value, Param
from thermo import Chemical
R = 8.314 # Gas constant J/(mol K)
Ea = 50000

def optimize_cstr(initial_conditions, rate_constant_298K, activation_energy, reactants, products):
    # Create a new model
    CSTR = ConcreteModel()

    # Constants
    R = 8.314 # Gas constant J/(mol K)
    
    # Define model variables
    CSTR.F_in_methanol = Var(domain=NonNegativeReals, initialize=initial_conditions['F_in_methanol'])
    CSTR.F_out_methyl_acetate = Var(domain=NonNegativeReals) # Initialize at 0 for the solver to find an optimum
    CSTR.V = Var(domain=NonNegativeReals, initialize=initial_conditions['V'])
    CSTR.T = Var(domain=NonNegativeReals, initialize=initial_conditions['T'])

    # Parameters for reaction rate
    CSTR.k0 = Param(initialize=rate_constant_298K)
    CSTR.Ea = Param(initialize=activation_energy)
    
    # Reaction rate expression using Arrhenius equation
    def reaction_rate_expression(model):
        return model.k0 * (Ea/(R * model.T)) * model.F_in_methanol#exp(-model.Ea / (R * model.T)) * model.F_in_methanol
    
    CSTR.reaction_rate = Var(domain=NonNegativeReals, initialize=0)
    CSTR.reaction_rate_constraint = Constraint(expr=CSTR.reaction_rate == reaction_rate_expression(CSTR))
    
    # Mole balance for methanol
    def mole_balance_rule(model):
        # Assuming a first-order reaction with respect to methanol, with no back-reaction
        return model.F_out_methyl_acetate == model.reaction_rate * model.V
    CSTR.mole_balance = Constraint(rule=mole_balance_rule)
    
    # Energy balance for the reactor
    def energy_balance_rule(model):
        # For simplicity, assume the heat capacity is constant and the reaction is isothermal
        # Here, delta_H_rxn would be obtained from thermochemical data for the reaction at T
        delta_H_rxn = reaction_enthalpy({'CH3OH': 1, 'CH3COOH': 1}, {'CH3COOCH3': 1, 'H2O': 1}, model.T)
        return 0 == -delta_H_rxn * model.reaction_rate * model.V
    CSTR.energy_balance = Constraint(rule=energy_balance_rule)
    
    # Objective function to maximize the conversion of methanol to methyl acetate
    def objective_rule(model):
        #Conversion X = (F_in - F_out) / F_in
        #Maximize F_out for the product, which is methyl acetate in this case
        return model.F_out_methyl_acetate
    CSTR.objective = Objective(rule=objective_rule, sense=1)  # sense=1 for maximization

    # Define a solver and solve the model
    solver = SolverFactory('ipopt') # 'ipopt' is an open-source solver for nonlinear optimization
    results = solver.solve(CSTR, tee=True)  # 'tee=True' to display solver output

    # Check if the solver found an optimal solution
    if (results.solver.status == 'ok') and (results.solver.termination_condition == 'optimal'):
        print('Found optimal solution.')
        print('F_in_methanol:', value(CSTR.F_in_methanol))
        print('F_out_methyl_acetate:', value(CSTR.F_out_methyl_acetate))
        print('V:', value(CSTR.V))
        print('T:', value(CSTR.T))
    else:
        print('No optimal solution found.')
    
    return CSTR

# Define a function to calculate the reaction enthalpy at a given temperature
def reaction_enthalpy(reactants, products, T):
    # For demonstration, using a placeholder for reaction enthalpy value which typically depends on T
    # In practice, this would require more complex calculations involving thermochemical databases
    return -50 * T # Placeholder value representing an exothermic reaction

# Example usage of the optimization function
initial_conditions = {
    'F_in_methanol': 1.0,  # Inlet flow of methanol
    'V': 10,              # Reactor volume
    'T': 350               # Temperature
}
rate_constant_298K = 1.0e4  # Example rate constant at 298K
activation_energy = 5000000    # Example activation energy in J/mol

# Optimize the CSTR
CSTR_model = optimize_cstr(initial_conditions, rate_constant_298K, activation_energy, reactants={}, products={})


In [ ]:
from pyomo.environ import ConcreteModel, Var, Piecewise, Constraint, Objective, SolverFactory, NonNegativeReals, exp, value
from thermo import Chemical

def optimize_cstr(initial_conditions, rate_constant_298K, activation_energy, reactants, products):
    CSTR = ConcreteModel()

    # Define model variables
    CSTR.F_in = Var(domain=NonNegativeReals, initialize=initial_conditions['F_in'])
    CSTR.V = Var(domain=NonNegativeReals, initialize=initial_conditions['V'])
    CSTR.T_in = Var(domain=NonNegativeReals, initialize=initial_conditions['T_in'])

    # Define specific heat, reaction enthalpy, and reaction rate functions
    def specific_heat(compound_formula, model):
        compound = Chemical(compound_formula)
        breakpoints = [compound.Tm, compound.Tb, 10000]  # Extended range with large number
        values = [
            compound.HeatCapacitySolid(compound.Tm) / compound.MW,
            compound.HeatCapacityLiquid(compound.Tm) / compound.MW,
            compound.HeatCapacityGas(compound.Tb) / compound.MW
        ]
        SH = Var(domain=NonNegativeReals)

        # Define the function rule separately
        def f_rule(model, x):
            if x <= breakpoints[0]:
                return values[0]
            elif x <= breakpoints[1]:
                return values[1]
            else:
                return values[2]

        Piecewise(SH, model.T_in,
                  pw_pts=breakpoints,
                  f_rule=f_rule,
                  pw_constr_type='EQ',  # yvar is equal to the piecewise function
                  pw_repn='SOS2')  # Use SOS2 as the piecewise representation method
        setattr(model, 'SH_C2H5O2C2H5', SH)
        return SH

    def reaction_enthalpy(reactants, products):
        delta_H_rxn = 0.0
        for reactant, coefficient in reactants.items():
            compound = Chemical(reactant)
            delta_H_rxn -= coefficient * compound.Hf
        for product, coefficient in products.items():
            compound = Chemical(product)
            delta_H_rxn += coefficient * compound.Hf
        return delta_H_rxn

    def reaction_rate(model):
        A_factor = rate_constant_298K * exp(-activation_energy / (8.314 * model.T_in))
        return A_factor * model.F_in

    # Initialize specific heat for C2H5O2C2H5
    CSTR.SH_C2H5O2C2H5 = specific_heat('C2H5O2C2H5', CSTR)

    # Define the energy balance constraint
    reactants_stoichiometry = {'C2H5O2C2H5': 1, 'NaOH': 1}
    products_stoichiometry = {'C2H3O2Na': 1, 'C2H5OH': 1}
    reaction_enthalpy_value = reaction_enthalpy(reactants_stoichiometry, products_stoichiometry)
    CSTR.energy_balance_constraint = Constraint(expr=CSTR.SH_C2H5O2C2H5 * CSTR.F_in - reaction_enthalpy_value * reaction_rate(CSTR) * CSTR.V == 0)

    # Define the objective function to maximize the production of sodium acetate
    CSTR.objective = Objective(expr=reaction_rate(CSTR) * CSTR.V, sense=-1)  # sense=-1 for maximization

    # Solve the optimization problem using a suitable solver
    solver = SolverFactory('gurobi')
    result = solver.solve(CSTR)

    # if result.solver.status == 'ok' and result.solver.termination_condition == 'optimal':
    #     print("Optimized Parameters:", value(CSTR.F_in), value(CSTR.V), value(CSTR.T_in))
    # else:
    #     print("No optimal solution found.")

    return result

# Example usage
initial_conditions = {'F_in': 1.0, 'V': 100, 'T_in': 350}
rate_constant_298K = 0.000001
activation_energy = 50000
CSTR_model = optimize_cstr(initial_conditions, rate_constant_298K, activation_energy)

In [ ]:
from pyomo.environ import ConcreteModel, Var, Piecewise, Objective, SolverFactory, NonNegativeReals, exp, value, Constraint
from thermo import Chemical

def optimize_cstr(initial_conditions, rate_constant_298K, activation_energy):
    CSTR = ConcreteModel()

    # Define model variables
    CSTR.F_in = Var(domain=NonNegativeReals,initialize=initial_conditions['F_in'])
    CSTR.F_out = Var(domain=NonNegativeReals)
    CSTR.V = Var(domain=NonNegativeReals, initialize=initial_conditions['V'])
    CSTR.T_in = Var(domain=NonNegativeReals, initialize=initial_conditions['T_in'])
    CSTR.T_out = Var(domain=NonNegativeReals)

    # Function to define specific heat using Piecewise
    def specific_heat(compound_formula, model):
        compound = Chemical(compound_formula)
        breakpoints = [compound.Tm, compound.Tb, 10000]  # Extended range with large number
        values = [
            compound.HeatCapacitySolid(compound.Tm) / compound.MW,
            compound.HeatCapacityLiquid(compound.Tm) / compound.MW,
            compound.HeatCapacityGas(compound.Tb) / compound.MW
        ]
        SH = Var(domain=NonNegativeReals)

        # Define the function rule separately
        def f_rule(model, x):
            if x <= breakpoints[0]:
                return values[0]
            elif x <= breakpoints[1]:
                return values[1]
            else:
                return values[2]

        Piecewise(SH, model.T_in,
                  pw_pts=breakpoints,
                  f_rule=f_rule,
                  pw_constr_type='EQ',  # yvar is equal to the piecewise function
                  pw_repn='SOS2')  # Use SOS2 as the piecewise representation method
        setattr(model, 'SH_C2H5O2C2H5', SH)
        return SH

    # Initialize specific heat for C2H5O2C2H5
    CSTR.SH_C2H5O2C2H5 = specific_heat('C2H5O2C2H5', CSTR)

    # Define the reaction enthalpy function within the main function
    def reaction_enthalpy(reactants, products):
        delta_H_rxn = 0.0
        for reactant, coefficient in reactants.items():
            compound = Chemical(reactant)
            delta_H_rxn -= coefficient * compound.Hf
        for product, coefficient in products.items():
            compound = Chemical(product)
            delta_H_rxn += coefficient * compound.Hf
        return delta_H_rxn

    # Define the reaction rate using the Arrhenius equation (assuming first-order kinetics)
    def reaction_rate(model):
        A_factor = rate_constant_298K * exp(-activation_energy / (8.314 * model.T_in))
        return A_factor * model.F_in

    # Define the energy balance constraint
    reactants_stoichiometry = {'C2H5O2C2H5': 1, 'NaOH': 1}
    products_stoichiometry = {'C2H3O2Na': 1, 'C2H5OH': 1}
    reaction_enthalpy_value = reaction_enthalpy(reactants_stoichiometry, products_stoichiometry)

    # Ensure the constraint uses the value at the current temperature
    CSTR.energy_balance_constraint = Constraint(expr=CSTR.SH_C2H5O2C2H5 * CSTR.F_in - reaction_enthalpy_value * reaction_rate(CSTR) * CSTR.V == 0)

    # Define the objective function to maximize the production of sodium acetate
    CSTR.objective = Objective(expr=CSTR.F_in - CSTR.F_out + reaction_rate(CSTR) * CSTR.V)#, sense=1)  # sense=1 for maximization

    # Solve the optimization problem using a suitable solver
    solver = SolverFactory('gurobi')
    #solver.options['mipgap'] = 0.01  # Optional: Set a gap tolerance if needed
    result = solver.solve(CSTR)#, tee=True, symbolic_solver_labels=True, keepfiles=False, mipgap=0.01)

    if result.solver.status == 'ok' and result.solver.termination_condition == 'optimal':
        print("Optimized Parameters:", value(CSTR.F_in), value(CSTR.V), value(CSTR.T_in))
    else:
        print("No optimal solution found.")

    return CSTR

# Example usage
initial_conditions = {
    'F_in': 1.0,  # Total inlet flow rate, arbitrary units
    'V': 100,    # Reactor volume, arbitrary units
    'T_in': 350   # Inlet temperature, K
}
rate_constant_298K = 0.000001  # Reaction rate constant at 298 K, arbitrary units
activation_energy = 50000  # Activation energy, J/mol

CSTR_model = optimize_cstr(initial_conditions, rate_constant_298K, activation_energy)


In [ ]:
from pyomo.environ import ConcreteModel, Var, Piecewise, Objective, SolverFactory, NonNegativeReals, exp, value, Constraint
from thermo import Chemical

def optimize_cstr(initial_conditions, rate_constant_298K, activation_energy):
    CSTR = ConcreteModel()

    # Define model variables
    CSTR.F_in = Var(domain=NonNegativeReals,initialize=initial_conditions['F_in'])
    CSTR.F_out = Var(domain=NonNegativeReals)
    CSTR.V = Var(domain=NonNegativeReals, initialize=initial_conditions['V'])
    CSTR.T_in = Var(domain=NonNegativeReals, initialize=initial_conditions['T_in'])

    # Function to define specific heat using Piecewise
    def specific_heat(compound_formula, model):
        compound = Chemical(compound_formula)
        breakpoints = [compound.Tm, compound.Tb, 10000]  # Extended range with large number
        values = [
            compound.HeatCapacitySolid(compound.Tm) / compound.MW,
            compound.HeatCapacityLiquid(compound.Tm) / compound.MW,
            compound.HeatCapacityGas(compound.Tb) / compound.MW
        ]
        SH = Var(domain=NonNegativeReals)

        # Define the function rule separately
        def f_rule(model, x):
            if x <= breakpoints[0]:
                return values[0]
            elif x <= breakpoints[1]:
                return values[1]
            else:
                return values[2]

        Piecewise(SH, model.T_in,
                  pw_pts=breakpoints,
                  f_rule=f_rule,
                  pw_constr_type='EQ',  # yvar is equal to the piecewise function
                  pw_repn='SOS2')  # Use SOS2 as the piecewise representation method
        setattr(model, 'SH_C2H5O2C2H5', SH)
        return SH

    # Initialize specific heat for C2H5O2C2H5
    CSTR.SH_C2H5O2C2H5 = specific_heat('C2H5O2C2H5', CSTR)

    # Define the reaction enthalpy function within the main function
    def reaction_enthalpy(reactants, products):
        delta_H_rxn = 0.0
        for reactant, coefficient in reactants.items():
            compound = Chemical(reactant)
            delta_H_rxn -= coefficient * compound.Hf
        for product, coefficient in products.items():
            compound = Chemical(product)
            delta_H_rxn += coefficient * compound.Hf
        return delta_H_rxn

    # Define the reaction rate using the Arrhenius equation (assuming first-order kinetics)
    def reaction_rate(model):
        A_factor = rate_constant_298K * exp(-activation_energy / (8.314 * model.T_in))
        return A_factor * model.F_in

    # Define the energy balance constraint
    reactants_stoichiometry = {'C2H5O2C2H5': 1, 'NaOH': 1}
    products_stoichiometry = {'C2H3O2Na': 1, 'C2H5OH': 1}
    reaction_enthalpy_value = reaction_enthalpy(reactants_stoichiometry, products_stoichiometry)

    # Ensure the constraint uses the value at the current temperature
    CSTR.energy_balance_constraint = Constraint(expr=CSTR.SH_C2H5O2C2H5 * CSTR.F_in - reaction_enthalpy_value * reaction_rate(CSTR) * CSTR.V == 0)

    # Define the objective function to maximize the production of sodium acetate
    CSTR.objective = Objective(expr=((CSTR.F_in - CSTR.F_out)/CSTR.F_out))

    # Solve the optimization problem using a suitable solver
    solver = SolverFactory('gurobi')
    #solver.options['mipgap'] = 0.01  # Optional: Set a gap tolerance if needed
    result = solver.solve(CSTR)#, tee=True, symbolic_solver_labels=True, keepfiles=False, mipgap=0.01)

    if result.solver.status == 'ok' and result.solver.termination_condition == 'optimal':
        print("Optimized Parameters:", value(CSTR.F_in), value(CSTR.V), value(CSTR.T_in))
    else:
        print("No optimal solution found.")

    return CSTR

# Example usage
initial_conditions = {
    'F_in': 1.0,  # Total inlet flow rate, arbitrary units
    'V': 100,    # Reactor volume, arbitrary units
    'T_in': 350   # Inlet temperature, K
}
rate_constant_298K = 0.000001  # Reaction rate constant at 298 K, arbitrary units
activation_energy = 50000  # Activation energy, J/mol

CSTR_model = optimize_cstr(initial_conditions, rate_constant_298K, activation_energy)
